**Aeronautics Institute of Technology – ITA**

**Computer Vision – CM-203**

**Professors:** 

Marcos Ricardo Omena de Albuquerque Maximo

Gabriel Adriano de Melo


**Instructions:**

Before submitting your lab, be sure that everything is running correctly (in sequence): first, **restart the kernel** (`Runtime->Restart Runtime` in Colab or `Kernel->Restart` in Jupyter). Then, execute all cells (`Runtime->Run All` in Colab or `Cell->Run All` in Jupyter) and verifies that all cells run without any errors, expecially the automatic grading ones, i.e. the ones with `assert`s.

**Do not delete the answer cells**, i.e. the ones that contains `WRITE YOUR CODE HERE` or `WRITE YOUR ANSWER HERE`, because they contain metadata with the ids of the cells for the grading system. For the same reason, **do not delete the test cells**, i.e. the ones with `assert`s. The autograding system executes all the code sequentially, adding extra tests in the test cells. There is no problem in creating new cells, as long as you do not delete answer or test cells. Moreover, keep your solutions within the reserved spaces.

The notebooks are implemented to be compatible with Google Colab, and they install the dependencies and download the datasets automatically. The commands which start with ! (exclamation mark) are bash commands and can be executed in a Linux terminal.

---

# Laboratório 8 - Segmentação Semântica

Neste laboratório exploraremos a segmentação semântica, isto é, a classificação de cada pixel de uma imagem. Isso ainda é uma etapa anterior à segmentação de instância, que além de classificar da pixel, atribui ele a uma instância específica (se houvessem vários objetos colados um no outro haveria distinção de cada instância de objeto).

Assim este laboratório está mais simples do que a de detecção de objetos (nanoYOLO, lab6). Iniciaremos com o Data Augmentation, que ficou faltando no lab6; definiremos a arquitetura da rede de segmentação, que apresenta um bottleneck e conexões residuais, tanto para o encoder quanto para o decoder; definiremos a função de loss, que é mais simples e finalmente realizaremos o treinamento completo, usando o transfer learning nos pesos do encoder de uma rede já treinada em classificação (de uma imagem inteira).

## Imports and Data Downloading

Todos os nossos imports serão realizados automaticamente pela FastAI. Enquanto ma maior parte das bibliotecas pode ser uma má ideia utilizar um star import (`*`) pois irá poluir o seu ambiente, o FastAI toma cuidado de limitar o que é exportado (importado pelo `*`) por meio da definição da variável `__all__` dentro de um módulo. Assim, tudo que é exportado/importado por ele é intencional.

In [ ]:
import pylab
import albumentations as A
from fastai.vision.all import *
from torchvision.ops import Conv2dNormActivation, SqueezeExcitation

Atenção, se por algum motivo você for mudar a pasta `/content` ou a flag de `RETRAIN`, mude na célula abaixo, pois ela é sobrescrita durante a correção (*read_only*) uma vez que para a correção funcionar o `base_path` tem que apontar para a pasta correta. Durante a correção, o `RETRAIN` será `False`.

In [ ]:
# dataset_lab read_only

! [ ! -d "/content/segmenta" ] && gdown -O /content/segmenta.zip "1H2JlJ3mDq9a4wSXNRruDl-0fO42yzeTh" &&  unzip -q /content/segmenta.zip -d /content && rm /content/segmenta.zip
! [ ! -d "/content/segmenta" ] && wget -P /content/ "http://ia.gam.dev/cm203/23/lab08/segmenta.zip"  &&  unzip -q /content/segmenta.zip -d /content && rm /content/segmenta.zip
base_path = Path("/content/segmenta")
%cd /content

RETRAIN = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Data Augmentation (2 pontos)

**Explicação sobre o assunto**

Apesar do FastAI já ter várias implementações internas de Data Augmentations (rotação, translação, escala, flip, entre outros) vamos utilizar uma outra biblioteca chamada [Albumentations](https://albumentations.ai), que apresenta ainda mais transformações. Contudo, essa biblioteca só opera a nível de matrizes (tensores) do formato Numpy, e não em tensores de PyTorch, nem em GPU, apenas CPU.

Lembre-se que em um dataset de segmentação cada pixel tem a sua própria anotação individual, e isso é guardado em uma outra matriz (que é uma imagem dos labels)

![Segmentation with labels](https://www.jeremyjordan.me/content/images/2018/05/Screen-Shot-2018-05-16-at-9.36.38-PM.png)

Vamos baixar os dados de imagens gravadas a partir da frente de um carro (tal qual se fosse um carro autônomo). Esse é o dataset [Cambridge-driving Labeled Video Database (CamVid)](http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/). Para este lab usaremos uma versão com resolução menor (tiny) para fins de treino mais rápido.

In [ ]:
data_path = untar_data(URLs.CAMVID)

As imagens tem as suas respectivas labels (anotações) de segmentação, vejamos os nomes de cada uma delas nesse dataset (atenção, não confunda com a da ilustração acima):

In [ ]:
nomes_labels = np.loadtxt(data_path/'codes.txt', dtype=str)
len(nomes_labels), nomes_labels

Veja um dataloader de segmentação criado usando a API de DataBlock do FastAI. Atente-se que a diferença maior agora é o `MaskBlock` que é justamente uma 'imagem' com os labels. Perceba que nesse dataset elas ficaram em outra pasta chamada `labels`, que nós precisamos acessa no `get_y`.

In [ ]:
camvid = DataBlock(
    blocks=(ImageBlock, MaskBlock(nomes_labels)),
    get_items=get_image_files,
    splitter=RandomSplitter(),
    get_y=lambda o: o.parent.parent/'labels'/f'{o.stem}_P{o.suffix}'
).dataloaders(data_path/"images", path=data_path, bs=8)

if RETRAIN:
    camvid.show_batch()

Observe a largura das imagens e de suas respectivas máscaras de segmentação $(960, 720)$. Veja como é o shape dos tensores respectivamente das imagens e das máscaras. Observe que a máscara é composta por números inteiros positivos e sequenciais que representam a classe do pixel. Durante a aplicação da loss function, teremos que convertê-los para uma representação de one-hot encoding.

In [ ]:
imagems, mascaras_segmenta = camvid.one_batch()
imagems.shape, mascaras_segmenta.shape, mascaras_segmenta[0]

Agora vamos ver as transformações que podem ser aplicadas às imagens e às anotações de segmentação.

Como as transformações são aplicadas antes do batch, vamos carregar uma imagem manualmente em conjunto com o seu label:

In [ ]:
img0 = PILImage.create(data_path/'images'/'0016E5_06660.png')
label0 = PILImage.create(data_path/'labels'/'0016E5_06660_P.png')

if RETRAIN:
    plt.imshow(np.array(label0)[..., 0], cmap='gist_ncar')
    plt.colorbar()
img0 if RETRAIN else None

Veja agora como aplicamos uma transformação da biblioteca Albumentations: 

Primeiro criamos a transformação com os parâmetros (limites e probabilidades) escolhidos; depois a aplicamos sobre a imagem e sobre as anotações de segmentação, simultaneamente, para que a mesma transformação (mesmos valores aleatórios) sejam aplicados igualmente sobre ambos. Caso contrário, poderíamos ter uma valor aleatório diferente de rotação entre a sua imagem e a sua anotação de segmentação. Para isso usamos atributos com nome `image` para a imagem original e `mask` para as anotações de segmentação. Essa distinção é importante, pois há transformações (de cores, por exemplo) onde apenas a imagem é alterada.

O resultado é um dicionário que contém as keys de `image` e de `mask` também, que podemos acessar. O formato de entrada e de saída são imagens (matrizes) Numpy.

In [ ]:
transform1 = A.ShiftScaleRotate(always_apply=True)
res = transform1(image=np.array(img0), mask=np.array(label0))

if RETRAIN:
    fig, axs = plt.subplots(1, 2, figsize=(12,5))
    axs[0].imshow(res['image'])
    axs[1].imshow(res['mask'][..., 0], cmap='gist_ncar')

Além disso, podemos compor várias transformações (como se fosse um Pipeline do FastAI) por meio do `Compose`. Olhe também outras transformações que não existem no FastAI, como o `RandomFog`, o `RandomRain`, o `RandomSunFlare` entre outras.

Execute várias vezes, e veja o sol, a chuva, as cores que variam um pouco.

In [ ]:
transform_compose = A.Compose([
    A.RandomCrop(width=512, height=512, always_apply=True),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(always_apply=True),
    A.ElasticTransform(),
    A.RandomBrightnessContrast(),
    A.RandomSunFlare(src_radius=90),
    A.RandomFog(fog_coef_upper=1),
    A.RandomRain(blur_value=2)
])

res2 = transform_compose(image=np.array(img0), mask=np.array(label0))

if RETRAIN:
    fig, axs = plt.subplots(1, 2, figsize=(12,5))
    axs[0].imshow(res2['image'])
    axs[1].imshow(res2['mask'][..., 0], cmap='gist_ncar')

**Enunciado da Questão**

Implemente a função `encodes` abaixo, de acordo com a sua documentação. A ideia é aplicar as trasformações do `NossoTransformaAugment`, declaradas na sua inicialização. Perceba que essa é uma classe do tipo `ItemTransform` que será utilizada posteriormente no `item_tfms` do `DataBlock` durante o treino de verdade.

O uso do Albumentations nessa aplicação foi apenas didático (não chove e está sempre bem iluminado), mas na vida real pode ser útil, por exemplo, veja mais exemplos de [Augmentations neste link](https://albumentations.ai/docs/getting_started/transforms_and_targets/)


**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas ([PyTorch](https://pytorch.org/docs/stable/), [FastAI](https://docs.fast.ai/), [Albumentations](https://albumentations.ai/docs/) mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>

Lembrando dos labs anteriores, utilize `np.array` para converter uma imagem do tipo PIL para uma matriz NumPy. Utilize `PILImage.create` e `PILMask.create` para fazer o inverso, lembrando que esses são tipos do FastAI.

</p>
</details>

In [ ]:
# questao_augmentation autograded_answer

class NossoTrasformaAugment(ItemTransform):
    split_idx = 0
    
    def __init__(self):
        self.tfm1 = A.HorizontalFlip(p=0.5)
        self.tfm2 = A.RandomBrightnessContrast(p=0.9)
        self.tfm3 = A.ShiftScaleRotate(always_apply=True)
        self.tfm4 = A.RandomRain(p=0.1, blur_value=1)
        self.transforma_total = A.Compose([self.tfm1, self.tfm2, self.tfm3, self.tfm4])

    def encodes(self, x):
        """Aplica as transformações tfm1/2/3 sequencialmente e igualmente para a
        imagem original e para as anotações de segmentação. Cada transformação
        poderá ser chamada apenas uma única vez.
        
        Args:
            x: Tupla com imagem original e imagem de anotações segmentadas
                tuple[PILImage, PILMask]

        Returns:
            Tupla com a imagem do tipo PILImage e PILMask
        """
        # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
        raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Vamos começar o teste com uma imagem mais de zeros apenas para verificar se os valores de retorno estão condizentes com o esperado:

In [ ]:
# testa_1_questao_augmentation autograder_tests 0.3

tfm_aug1 = NossoTrasformaAugment()
pil_img_zeros = PILImage.create(np.zeros((100, 100, 3), dtype=np.uint8))
pil_mask_zeros = PILMask.create(np.zeros((100, 100), dtype=np.uint8))

res_encode = tfm_aug1.encodes((pil_img_zeros, pil_mask_zeros))

assert type(res_encode) is tuple
assert len(res_encode) == 2
assert type(res_encode[0]) is PILImage
assert type(res_encode[1]) is PILMask

Vamos verificar se você chamou na ordem correta e se foram todas as transformações mesmo. Como os valores são pseudo-aleatórios, vamos reinicializar os estados (seed) dos geradores de números pseudo-aleatórios (a função `set_seed` vem do FastAI e ela age sobre o PyTorch, Numpy, Python), para que haja reproducidibilidade:

In [ ]:
# testa_2_questao_augmentation autograder_tests 1.7

set_seed(2023_10_13, reproducible=True)

tfm_aug2 = NossoTrasformaAugment()
img_t2 = PILImage.create(data_path/'images'/'0016E5_06660.png')
label_t2 = PILImage.create(data_path/'labels'/'0016E5_06660_P.png')

res_img2, res_label2 = tfm_aug2.encodes((img_t2, label_t2))

assert np.array(res_img2).shape == (720, 960, 3)
assert res_label2.shape == (720, 960)
assert np.all(np.array(res_img2).sum(axis=(0,1)) == [47285429, 49826794, 52152949])
assert abs(np.array(res_label2).mean() - 19.371778) < 1e-5


Veja o resultado:

In [ ]:
res_img2 if RETRAIN else None

## Arquitetura do Encoder  (2 pontos)

**Explicação sobre o assunto**

Nós poderíamos utilizar diretamente o `unet_learner` do FastAI, que já faria automaticamente toda a definição da arquitetura baseada na Unet com encoder, decoder e loss-function adequadas à segmentação. Contudo, vamos realizar a definição de cada uma dessas partes separadamente, para que você possa entendê-la mais profundamente e também aplicar os conceitos de transfer-learning, além de continuar se familiarizando com o framework do PyTorch e com implementações de redes neurais.

A arquitetura do encoder nada mais é do que a mesma que encontramos em redes de classificação de imagens. Nós vamos aproveitar justamente o fato de que na maioria dessas arquitetura há diminuição das dimensões espaciais, em geral, em um fator de 2 a cada módulo, conforme observado na Figura abaixo. Na UNet do paper original eles não utilizavam padding, e a saída ficava com um tamanho espacial diferente da entrada, mas nas implementações posteriores, tornou-se muito comum utilizar padding same nas convoluções.

![Arquitetura baseada em UNet](https://nchlis.github.io/2019_10_30/architecture_unetV2.png)

Em vez de utilizar uma arquitetura já implementada nos labs anteriores (ResNet ou ResNext) vamos para outra um pouco mais recente: MobileNet-v3. É de 2019 (que ainda é bem antigo) mas é bem mais recente do que as ResNext (2016).

Vejamos os seus parâmetros:

In [ ]:
pesos_mnv3 = MobileNet_V3_Large_Weights.IMAGENET1K_V2.get_state_dict(progress=True)
list(pesos_mnv3.keys()), pesos_mnv3

Assim, vamos definir abaixo a arquitetura da rede, que é simplesmente a aplicação sequencial de blocos menores. Um dos 'segredos' da MobileNet está justamente na definição desses hyperparâmetros, que foram selecionados automaticamento pelo treino de milhares de redes em paralelo com diferentes combinações (AutoML).

Todos esses blocos convolucionais são agrupados sequencialmente no módulo `self.features`. Em seguida temos um global average pooling seguido pela aplicação de uma rede completamente conectada (densa, *MLP*) `self.classifier`.

Em suma, essa é a arquitetura da MobileNet, apresentada no paper [Searching for MobileNetV3](https://arxiv.org/abs/1905.02244), cujo código está implementado no próprio PyTorch. O código abaixo foi retirado da implementação padrão.

Para mais alguns detalhes, observe que o batch_norm2d foi chamado por meio da função built-in do Python `partial` para passar os parâmetros de momento e de EPS (que é o número que se soma ao divisor para evitar divisão por zero). A convolução é seguida diretamente pela camada de normalização e pela função de ativação, tudo isso embutido na camada `Conv2dNormActivation`. Utiliza-se a função de ativação `Hardswish`.

In [ ]:
class MobNet3large(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        batch_norm2d = partial(nn.BatchNorm2d, eps=0.001, momentum=0.01)
        
        # Camada de Entrada
        camadas = [Conv2dNormActivation(3, 16, kernel_size=3, stride=2,
                                norm_layer=batch_norm2d, activation_layer=nn.Hardswish)]
        
        # Blocos
        hyperparams_blocks = [(16,  3,  16,  16, False, False, 1),
                              (16,  3,  64,  24, False, False, 2),
                              (24,  3,  72,  24, False, False, 1),
                              (24,  5,  72,  40,  True, False, 2),
                              (40,  5, 120,  40,  True, False, 1),
                              (40,  5, 120,  40,  True, False, 1),
                              (40,  3, 240,  80, False,  True, 2),
                              (80,  3, 200,  80, False,  True, 1),
                              (80,  3, 184,  80, False,  True, 1),
                              (80,  3, 184,  80, False,  True, 1),
                              (80,  3, 480, 112,  True,  True, 1),
                              (112, 3, 672, 112,  True,  True, 1),
                              (112, 5, 672, 160,  True,  True, 2),
                              (160, 5, 960, 160,  True,  True, 1),
                              (160, 5, 960, 160,  True,  True, 1)]
        for hparam in hyperparams_blocks:
            camadas.append(MN3Block(hparam, batch_norm2d))
        
        # Convolução de Saída
        camadas.append(Conv2dNormActivation(160, 960, kernel_size=1,
                                norm_layer=batch_norm2d, activation_layer=nn.Hardswish))
        self.features = nn.Sequential(*camadas)
        self.classifier = nn.Sequential(nn.Linear(960, 1280),
                                        nn.Hardswish(inplace=True),
                                        nn.Dropout(p=0.2, inplace=True),
                                        nn.Linear(1280, num_classes))

    def forward(self, x: Tensor) -> Tensor:
        x = self.features(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

Observe que o bloco nós chamamos de `MN3Block` e será implementado por você a seguir, ele recebe uma tupla que define os seus hyper-parâmetros: *in_chan, kernel, expanded_chan, out_chan, use_se, use_hs, stride*

**Enunciado da Questão**

Implemente a função `forward` da classe `MN3Block` abaixo, de acordo com a sua documentação. A ideia é aplicar todos os módulos sequencialmente e, se houver conexão residual, aplicá-la também.

Observe a figura abaixo com os quatro módulos (convoluções) que são aplicados: 
- a convolução de entrada apenas sobre os canais (`kernel_size=1`)
- a convolução apenas espacial, separada para cada canal (`groups=channels`)
- a camada de Squeeze-and-Excite que simplemente multiplica cada canal por um escalar diferente (definido por uma rede densa)
- a camada de saída convolucional, que assim como a entrada, atua apenas sobre os canais (`kernel_size=1`)

![Mobile Net V3](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-21_at_11.03.15_PM.png)


**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>
Use o `self.block` para chamar todos os módulos sequencialmente.

Lembre-se que para implementar uma conexão residual basta somar a entrada (antes de realizar qualquer operação) à saída. Nesse caso as dimensões já estão casadas.
</p>
</details>

In [ ]:
# questao_encoder autograded_answer

class MN3Block(nn.Module):
    def __init__(self, hyperparams, norm_layer):
        super().__init__()
        in_chan, kernel, expanded_chan, out_chan, use_se, use_hs, stride = hyperparams
        act_layer = nn.Hardswish if use_hs else nn.ReLU
        se_layer = partial(SqueezeExcitation, scale_activation=nn.Hardsigmoid)
        lista = []
        
        # input to expanded (channels)
        if expanded_chan != in_chan:
            lista.append(Conv2dNormActivation(in_chan, expanded_chan, kernel_size=1,
                            norm_layer=norm_layer, activation_layer=act_layer))
        
        # depthwise convolution
        lista.append(Conv2dNormActivation(expanded_chan, expanded_chan, kernel_size=kernel,
            stride=stride, groups=expanded_chan, norm_layer=norm_layer, activation_layer=act_layer))
        
        # Squeeze-and-Excite Layer
        if use_se:
            squeeze_chan = int((expanded_chan // 4) + 4) // 8 * 8
            squeeze_chan += 8 if squeeze_chan < 0.9 * (expanded_chan // 4) else 0
            lista.append(se_layer(expanded_chan, squeeze_chan))
        
        # expanded to output (channels)
        lista.append(Conv2dNormActivation(expanded_chan, out_chan, kernel_size=1,
                                           norm_layer=norm_layer, activation_layer=None))
        
        self.block = nn.Sequential(*lista)
        self.existe_conexao_residual = (stride == 1 and in_chan == out_chan)

    def forward(self, x: Tensor) -> Tensor:
        """ Realiza a propagação direta do bloco
        Aplique todas as camadas sequencialmente
        Se houver conexão residual, aplique-a também

        Args:
            x: tensor de entrada ao bloco de shape (B, in_chan, H, W)

        Returns:
            tensor com shape (B, out_chan, H, W)
        """
        # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
        raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Verifica se o shape de saída está correto:

In [ ]:
# testa_1_questao_encoder autograder_tests 0.1

bl0 = MN3Block((16,  3,  32,  32, False, False, 1), nn.BatchNorm2d)
x0 = torch.randn((3, 16, 7, 7))

out0 = bl0(x0)
assert out0.shape == (3, 32, 7, 7)


Verifica se o código ainda funciona quando há conexão residual:

In [ ]:
# testa_2_questao_encoder autograder_tests 0.1

bl1 = MN3Block((16,  3,  16,  16, False, False, 1), nn.BatchNorm2d)
x1 = torch.randn((3, 16, 7, 7))

out1 = bl1(x1)
assert out1.shape == (3, 16, 7, 7)


Verifica se a implementação está correta, comparando com a implementação padrão do PyTorch (OBS: é importantíssimo utilizar o `.train(False)` para reconfigurar o dropout e o batch_norm para inferência, senão eles agiriam de forma aleatória):

In [ ]:
# testa_3_questao_encoder autograder_tests 1.8

pesos_mnv3 = MobileNet_V3_Large_Weights.IMAGENET1K_V2.get_state_dict(progress=True)

mn3l = MobNet3large()
mn3l.load_state_dict(pesos_mnv3)
mn3l.train(False)

padrao = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V2)
padrao.train(False)

x3 = torch.randn((1, 3, 256, 256))

with torch.no_grad():
  out3 = mn3l(x3)
  saida_padrao = padrao(x3)

assert torch.linalg.norm(out3 - saida_padrao) < 1e-5


## Arquitetura do Decoder  (2 pontos)

**Explicação sobre o assunto**

Agora vamos codificar o decoder, que irá receber como entrada o espaço latente codificado pelo encoder, isto é, o tensor com dimensões espaciais reduzidas (mas com mais canais).

Enquanto nas arquiteturas mais usuais, há vários estágios para o upsampling, isto é, para o aumento das dimensões espaciais, neste, nós usaremos a arquitetura do DeepLabV3, do artigo [Rethinking Atrous Convolution for Semantic Image Segmentation](https://arxiv.org/abs/1706.05587). Em poucas palavras, ele utiliza várias convoluções em paralelo, alterando-se a dilatação dessas convoluções, tudo isso encapsulado no módulo de *Atrous Spatial Pyramid Pooling* (ASPP).

Abaixo temos o código desse *encoder*, que é simplemente composto de convoluções que não alteram as dimensões espaciais.

In [ ]:
conv2d_bnrelu = partial(Conv2dNormActivation, norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU, bias=False)

class DeepLabDecoder(nn.Sequential):
    def __init__(self, in_channels: int, num_classes: int):
        super().__init__(AtrousSpatialPyramidPooling(in_channels),
                         conv2d_bnrelu(256, 256, kernel_size=3, padding=1),
                         nn.Conv2d(256, num_classes, kernel_size=1))

É importante resaltar que nas operações de UpSampling, recomenda-se que o parâmetro `align_corners` seja `False` para evitar enviesamento da posição, como se observa na figura abaixo. Se a posição ficar enviesada, as camadas convolucionais perdem a sua propriedade de equivariância à translação.

![Align Corners](https://discuss.pytorch.org/uploads/default/original/2X/6/6a242715685b8192f07c93a57a1d053b8add97bf.png)

Assim, veja abaixo a implementação do módulo `ASPPPooling` que faz um global average pooling (média sobre todas as dimensões espaciais), usa uma camada convolucional e faz um upsampling (repete os valores) para retornar o mesmo tamanho (dimensões espaciais) da entrada.

In [ ]:
class ASPPPooling(nn.Sequential):
    def __init__(self, in_chan: int, out_chan: int):
        super().__init__(nn.AdaptiveAvgPool2d(1),
                         conv2d_bnrelu(in_chan, out_chan, kernel_size=1))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        size = x.shape[-2:]
        return F.interpolate(super().forward(x), size=size, mode="bilinear", align_corners=False)

**Enunciado da Questão**

Implemente a função `forward` da classe `AtrousSpatialPyramidPooling` abaixo, de acordo com a sua documentação. A ideia é aplicar as convoluções (`self.convs`, cada uma com sua própria dilatação, assim forma uma pirâmide) todas em paralelo, depois concatene todos esses valores na camada dos canais e utilize a convolução final.

![Atrous Spatial Pyramid Pooling](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-28_at_2.59.27_PM.png)

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>
Utilize um for-loop para percorrer todas as convs e vá salvando o resultado em uma lista Python.

Use a função `torch.cat` para concatenar os tensores. Utilize a dimensão correta (dos canais).
</p>
</details>

In [ ]:
# questao_decoder autograded_answer

class AtrousSpatialPyramidPooling(nn.Module):
    def __init__(self, in_chan: int, out_chan=256):
        super().__init__()
        lista = [conv2d_bnrelu(in_chan, out_chan, kernel_size=1)]
        for r in (12, 24, 36):
            lista.append(conv2d_bnrelu(in_chan, out_chan, kernel_size=3, padding=r, dilation=r))
        lista.append(ASPPPooling(in_chan, out_chan))

        self.convs = nn.ModuleList(lista)

        self.final = nn.Sequential(conv2d_bnrelu(len(lista) * out_chan, out_chan, kernel_size=1),
                                   nn.Dropout(0.5))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """ Realiza a propagação direta do bloco
        Aplique as camadas convs todas paralelamente.
        Concatene os resultados dessas convs (na dimensão dos canais)
        Aplique a convolução final sobre o tensor concatenado

        Args:
            x: tensor de entrada ao bloco de shape (B, in_chan, H, W)

        Returns:
            tensor com shape (B, out_chan, H, W)
        """
        # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
        raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Verifica se o shape de saída está correto:

In [ ]:
# testa_1_questao_decoder autograder_tests 0.2

aspp0 = AtrousSpatialPyramidPooling(128, 384)

x0 = torch.randn((3, 128, 64, 64))

out0 = aspp0(x0)
assert out0.shape == (3, 384, 64, 64)


Verifica se a implementação está correta, comparando com o resultado esperado:

In [ ]:
# testa_2_questao_decoder autograder_tests 1.8

aspp1 = AtrousSpatialPyramidPooling(64, 128)
aspp1.load_state_dict(torch.load(base_path / 'aspp1.pth'))
aspp1.train(False)

x1 = torch.load(base_path / 'x1.pt')

out1 = aspp1(x1)
out1_esperado = torch.load(base_path / 'out1.pt')

assert out1.shape == (1, 128, 78, 79)
assert torch.linalg.norm(out1 - out1_esperado) < 1e-4


## Loss Function  (2 pontos)

**Explicação sobre o assunto**

$L = \frac{-1}{B H W} \sum_B \sum_H \sum_W \sum_C p_{b,h,w,c} \log{\hat{p}_{b,h,w,c}}$ , onde $B$ é a quantidade de dados no batch, $C$ é a quantidade de canais, $H$ e $W$ são a altura e largura, $p_{b,h,w,c}$ é a probabilidade verdadeira para o pixel específico no canal e batch definidos, e $\hat{p}_{b,h,w,c}$ é a probabilidade estimada.

In [ ]:
logits_estimados = torch.randn(size=(1, 17, 120, 130))
probabilidades_verdadeiras = F.softmax(torch.randn(size=(1, 17, 120, 130)), dim=1)

F.cross_entropy(logits_estimados, probabilidades_verdadeiras)

A função F.cross_entropy do PyTorch também aceita passar os labels verdadeiros (em vez das probabilidades verdadeiras). Imagine que isso simplesmente é antes de fazer o one-hot encoding dos labels.

In [ ]:
logits_estimados = torch.randn(size=(20, 17))
classes_verdadeiras = torch.randint(low=0, high=17, size=(20,))

F.cross_entropy(logits_estimados, classes_verdadeiras)

E novamente também ela está preparada para lidar com tipos de imagens, isto é, com shape da forma $(B, C, H, W)$.

In [ ]:
logits_estimados = torch.randn(size=(1, 17, 120, 130))
classes_verdadeiras = torch.randint(low=0, high=17, size=(1, 120, 130))

F.cross_entropy(logits_estimados, classes_verdadeiras)

**Enunciado da Questão**

Implemente a função `simple_segmentation_loss` abaixo, de acordo com a sua documentação. A ideia é implementar a entropia cruzada pixel-a-pixel (realizando a média de todos esses valores, seja dos pixels ou do batch).

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>
Use a função do PyTorch `F.cross_entropy`, preste atenção para ver se você não está chamando-a de forma errada.
</p>
</details>

In [ ]:
# questao_loss autograded_answer

def simple_segmentation_loss(ŷ: Tensor, y: Tensor) -> Tensor:
    """Retorna a loss function da entropia cruzada média (pixel-a-pixel) 
    
    Args:
        ŷ: Tensor dos logits dos labels, shape (B, C, H, W) ou (B, C)
        y: Tensor dos labels verdadeiros, shape (B, H, W) ou (B, )
    
    Returns:
        Tensor de rank 0 que é loss de segmentação
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Verifica se o shape de saída está correto:

In [ ]:
# testa_1_questao_loss autograder_tests 0.2

ŷ0 = torch.randn((1,30))
y0 = tensor([8])

loss0 = simple_segmentation_loss(ŷ0, y0)
assert type(loss0) == Tensor
assert loss0.shape == tuple()
assert loss0.item()

Verificando o valor para apenas um tensor simples:

In [ ]:
# testa_2_questao_loss autograder_tests 0.2

ŷ0 = tensor([[-3, 2, 3, 8.]])
y0 = tensor([3])

assert abs(simple_segmentation_loss(ŷ0, y0) - 0.0092) < 1e-4

Verificando o valor para o shape esperado de imagem:

In [ ]:
# testa_3_questao_loss autograder_tests 1.6

ŷ0 = tensor([-3, 2, 3, 8, 2, 3, 0, 6.]).reshape(1, 4, 2, 1)
y0 = tensor([1, 1]).reshape(1, 2, 1)

assert abs(simple_segmentation_loss(ŷ0, y0) - 0.2429) < 1e-4


## Arquitetura Final (2 pontos)

**Explicação sobre o assunto**


Após essas operações de convolução que não alteram as dimensões espaciais, há finalmente o upsampling para o tamanho final da imagem.

Essa arquitetura é bem simples, mas por causa desse upsampling brusco, peca ao perder os detalhes, principalmente de objetos pequenos ou de bordas. Contudo, por ter menos operações, é uma arquitetura relativamente rápida, enquanto consegue manter parte da acurácia.

A imagem abaixo representa esse salto do upsampling, embora no nosso caso seja de 16x (pois há 4 downsampling de 2x na MobileNetV3), o que é ainda mais brusco.

![DeepLab](https://www.imaios.com/i/var/site/storage/images/6/9/8/9/499896-2-eng-GB/14%20-%20Architecture%20%20Deeplab%20V3.png?ixlib=php-3.3.1&q=75&w=920)

**Enunciado da Questão**

Implemente a função `forward` da classe `DeepLab` abaixo, de acordo com a sua documentação. A ideia é juntar o que já foi implementado invocando o encoder, o decoder e fazendo o upsampling final.

Veja na imagem abaixo como ele representa: O `backbone` é o nosso encoder, as operações piramidais e convoluções já estão embutidas no nosso decoder, e o upsampling é realizado por uma interpolação bilinear não enviesada.

![DeepLab](https://www.imaios.com/i/var/site/storage/images/7/8/8/9/499887-1-eng-GB/Atrous-Spatial-Pyramid-Pooling-Module.png?ixlib=php-3.3.1&q=75&w=565)

Essa questão acabou que ficou dependendo da questão do decoder.

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>
Basta chamar o encoder e depois o decoder.

Utilize a função `F.interpolate` para fazer o upsampling final. Veja como ela é chamada lá acima, na classe do `ASPPPooling`.
</p>
</details>

In [ ]:
# questao_arq autograded_answer

class DeepLab(nn.Module):
    def __init__(self, num_classes: int, mobile_net=None):
        super().__init__()
        if mobile_net is None:
            mobile_net = MobNet3large()
            mobile_net.load_state_dict(MobileNet_V3_Large_Weights.IMAGENET1K_V2.get_state_dict(progress=True))
        self.encoder = mobile_net.features
        self.decoder = DeepLabDecoder(960, num_classes)

    def forward(self, x: Tensor) -> Tensor:
        """ Realiza a propagação direta do bloco:
        Aplique o encoder, em seguida o decoder,
        Aplique o upsampling final, interpolação bilinear não enviesada.

        Args:
            x: tensor de entrada ao bloco de shape (B, 3, H, W)

        Returns:
            tensor com shape (B, num_classes, H, W)
        """
        # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
        raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Verifica se o shape de saída está correto:

In [ ]:
# testa_1_questao_arq autograder_tests 0.2

mobnet = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V2)
dlseg = DeepLab(7, mobnet)

imgrand = torch.rand((5, 3, 100, 120))

out_rand = dlseg(imgrand)

assert out_rand.shape == (5, 7, 100, 120)

Agora vamos verificar se as operações foram implementadas corretamente. Vamos comparar com o resultado esperado de uma rede já treinada:

In [ ]:
# testa_2_questao_arq autograder_tests 1.8

mobnet = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V2)
dlseg = DeepLab(32, mobnet)

img0 = Normalize.from_stats(*imagenet_stats, cuda=False)(ToTensor()(
                PILImage.create(data_path/'images'/'0016E5_06660.png'))/255)

dlseg.load_state_dict(torch.load(base_path/'treinado.pth', 'cpu'))
dlseg.train(False)

with torch.no_grad():
    res = dlseg(img0)
res_esperado = torch.load(base_path/'res.pt')

assert torch.linalg.norm(res - res_esperado) < 1e-2


Veja o resultado da imagem inicial em uma rede já treinada:

In [ ]:
if RETRAIN:
    plt.figure(figsize=(10, 6))
    plt.imshow(PILImage.create(data_path/'images'/'0016E5_06660.png'))
    plt.imshow(res.numpy().argmax(axis=1)[0], cmap='gist_ncar', alpha=0.5)
    plt.axis('off')

## Treinando de Verdade (Use GPU)

**Atenção: Habilite a GPU no Colab para poder treinar em poucos minutos (ao invés de várias horas na CPU)**

No primeiro passo vamos redefinir o nosso dataloader, com atenção para realizar a normalização da imagem de acordo com o que foi utilizado no encoder já pré-treinado (no caso ele treinou no ImageNet que tinha uma média e um desvio padrão específicos).

In [ ]:
camvid_datablock = DataBlock(
    blocks=(ImageBlock, MaskBlock(nomes_labels)),
    get_items=get_image_files,
    splitter=RandomSplitter(),
    get_y=lambda o: o.parent.parent/'labels'/f'{o.stem}_P{o.suffix}',
    item_tfms=NossoTrasformaAugment(),
    batch_tfms=Normalize.from_stats(*imagenet_stats)
)
camvid = camvid_datablock.dataloaders(data_path/"images", path=data_path, bs=4)

Vamos ver as imagens geradas pelo dataloader (com as augmentations que fizemos na primeira questão):

In [ ]:
if RETRAIN:
    camvid.show_batch()

Agora vamos instanciar a nossa rede, que irá utilizar o que implementamos, seja a arquitetura total que tem o encoder e o decoder e também a nossa loss function que implementamos separadamente.

Perceba que nesse primeiro momento desabilitamos o treino do encoder (`requires_grad=False`):

In [ ]:
deeplab = DeepLab(len(nomes_labels))
deeplab.encoder.requires_grad = False

learner = Learner(camvid, deeplab, simple_segmentation_loss)

Veja o learning rate que seria adequado para iniciarmos o treinamento:

In [ ]:
if RETRAIN:
    learner.lr_find()

No meu caso, nós podemos nos aventurar mais do que ele sugeriu (mas tem que ser antes de começar a divergir pois o gráfico que ele mostra é suavizado).

Se quiser, você também pode treinar por mais de duas épocas:

In [ ]:
if RETRAIN:
    learner.fit_one_cycle(n_epoch=2, lr_max=0.01)

Agora vamos habilitar o treino do encoder também, otimizando a rede de ponta-a-ponta. Vejamos também onde fica o *cotovelo* no gráfico da learning rate (LR):

In [ ]:
deeplab.encoder.requires_grad = True
if RETRAIN:
    learner.lr_find()

Agora por segurança, vamos colocar uma LR de pelo menos uma ordem de grandeza antes do cotovelo:

In [ ]:
if RETRAIN:
    learner.fit_one_cycle(n_epoch=6, lr_max=0.001)

Em geral, pelo modelo ter apenas um único estágio de upsampling, fazendo um pulo brusco das dimensões espaciais, ele acaba perdendo os detalhes mais finos. O trade-off dele é ser bem mais leve (faz menos operações).

Vejamos os resultados qualitativamente (a loss function do FastAI é mais esperta e já faz o argmax na hora de exibir os resultados):

In [ ]:
if RETRAIN:
    learner_show = Learner(camvid, dlseg, CrossEntropyLossFlat(axis=1))
    learner_show.show_results(max_n=4)

Se quiser, pode salvar os pesos do seu modelo:

In [ ]:
if RETRAIN:
    torch.save(deeplab.state_dict(), base_path / 'treinado_meu.pth')

## One-liner com FastAI Segmentation Learner (Use GPU)

O FastAI já automaticamente utiliza a arquitetura completa baseada na UNET e com pesos do encoder já pré-treinados. Nesse caso específico, nós utilizamos a ResNet34 como encoder:

Você também pode treinar por mais épocas.

**Em caso de erro por falta de memória `RuntimeError: CUDA error: out of memory`, reinicie o kernel e não execute o treinamento da nossa rede acima (apenas até o dataloader)**

<sub>(Não se preocupe com warning, o FastAI está chamando uma função deprecada do PyTorch)</sub>

In [ ]:
if RETRAIN:
    learn_fastai = unet_learner(camvid, resnet34)
    learn_fastai.fine_tune(epochs=5)

Vejamos os resultados, qualitativamente:

In [ ]:
if RETRAIN:
    learn_fastai.show_results(max_n=4)

# Your data and feedback:

Write a feedback for the lab so we can make it better for the next years.

In the following variables, write the number of hours spent on this lab, the perceived difficulty, and the expected grade (you may delete the `raise` and the comments):

In [ ]:
# meta_eval manual_graded_answer 0

horas_gastas = None    # 1.5   - Float number with the number of hours spent 
dificuldade_lab = None # 0     - Float number from 0.0 to 10.0 (inclusive)
nota_esperada = None   # 10    - Float number from 0.0 to 10.0 (inclusive)

# WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
raise NotImplementedError()

Write below other comments or feedbacks about the lab. If you did not understand anything about the lab, please also comment here.

If you find any typo or bug in the lab, please comment below so we can fix it.

WRITE YOUR SOLUTION HERE! (do not change this first line):

**ATTENTION**

**ATTENTION**

**ATTENTION**

**ATTENTION**

**DISCURSIVE QUESTION**

WRITE YOUR ANSWER HERE (do not delete this cell so the ID is not lost)

**ATTENTION**

**ATTENTION**


**End of the lab!**